# Teacher vs Course Classification

Heuristics, baselines, transformers, robustness checks, and prompting for topic detection.

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

DATA_PATH = Path("data_feedback.xlsx")
RANDOM_SEED = 7

## 2. Load and inspect

In [ ]:
df = pd.read_excel(DATA_PATH).rename(columns={"teacher/course": "topic"})
print(df.head())
print(df["topic"].value_counts())

## 3. Split and helpers

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.25, stratify=df["topic"], random_state=RANDOM_SEED)

def evaluate(model, x_train, y_train, x_val, y_val, label):
    model.fit(x_train, y_train)
    preds = model.predict(x_val)
    print(f"=== {label} ===")
    print(classification_report(y_val, preds))
    cm = confusion_matrix(y_val, preds, labels=sorted(y_val.unique()))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=sorted(y_val.unique()), yticklabels=sorted(y_val.unique()))
    plt.title(label)
    plt.xlabel("Pred")
    plt.ylabel("True")
    plt.show()
    return preds

## 4. Heuristic baseline

In [ ]:
def heuristic_classifier(text):
    t = text.lower()
    if any(k in t for k in ["teacher", "sir", "madam", "he", "she"]):
        return "teacher"
    if any(k in t for k in ["course", "curriculum", "practical", "syllabus"]):
        return "course"
    return "teacher"
heuristic_preds = val_df["comments"].apply(heuristic_classifier)
print(classification_report(val_df["topic"], heuristic_preds))

## 5. TF-IDF baselines

In [ ]:
word_clf = Pipeline([("tfidf", TfidfVectorizer(ngram_range=(1,2))), ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))])
_ = evaluate(word_clf, train_df["comments"], train_df["topic"], val_df["comments"], val_df["topic"], "Word TF-IDF")
char_clf = Pipeline([("tfidf", TfidfVectorizer(analyzer="char", ngram_range=(3,5))), ("clf", LogisticRegression(max_iter=500, class_weight="balanced"))])
_ = evaluate(char_clf, train_df["comments"], train_df["topic"], val_df["comments"], val_df["topic"], "Char TF-IDF")

## 6. Sentiment-prefixed robustness check

In [ ]:
sent_pref_train = [f"[SENT={s}] {c}" for s, c in zip(train_df["sentiment"], train_df["comments"])]
sent_pref_val = [f"[SENT={s}] {c}" for s, c in zip(val_df["sentiment"], val_df["comments"])]
_ = evaluate(word_clf, sent_pref_train, train_df["topic"], sent_pref_val, val_df["topic"], "Sentiment-prefixed TF-IDF")

## 7. Transformer fine-tuning

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
label2id = {l:i for i,l in enumerate(sorted(df["topic"].unique()))}
id2label = {i:l for l,i in label2id.items()}
def tok(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)
train_ds = Dataset.from_dict({"text": train_df["comments"].tolist(), "label": [label2id[x] for x in train_df["topic"]]})
val_ds = Dataset.from_dict({"text": val_df["comments"].tolist(), "label": [label2id[x] for x in val_df["topic"]]})
train_ds = train_ds.map(tok, batched=True)
val_ds = val_ds.map(tok, batched=True)
train_ds.set_format("torch")
val_ds.set_format("torch")
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label2id), id2label=id2label, label2id=label2id)
args = TrainingArguments(output_dir="./topic_distilbert", evaluation_strategy="epoch", save_strategy="epoch", num_train_epochs=5, per_device_train_batch_size=8, per_device_eval_batch_size=16, learning_rate=2e-5, weight_decay=0.01, logging_steps=10)
trainer = Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=val_ds, tokenizer=tokenizer)
# trainer.train()
# trainer.evaluate()

## 8. Sentiment-shift stress (train positive only)

In [ ]:
positive_df = df[df["sentiment"] == "positive"]
shift_train, shift_val = train_test_split(positive_df, test_size=0.3, stratify=positive_df["topic"], random_state=RANDOM_SEED)
shift_clf = Pipeline([("tfidf", TfidfVectorizer(ngram_range=(1,2))), ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))])
shift_clf.fit(shift_train["comments"], shift_train["topic"])
robust_preds = shift_clf.predict(val_df["comments"])
print(classification_report(val_df["topic"], robust_preds))

## 9. Prompting baseline

In [ ]:
from transformers import pipeline
prompt_template = "Is this feedback about the teacher or the course? Respond with teacher or course only. Text: {text}"
zero_shot = pipeline("text-classification", model="facebook/bart-large-mnli")
print(zero_shot(prompt_template.format(text=val_df.iloc[0]["comments"])))

## 10. Error analysis

In [ ]:
def collect_errors(model, x_val, y_val):
    preds = model.predict(x_val)
    return pd.DataFrame({"text": x_val, "true": y_val, "pred": preds}).query("true != pred")
err_df = collect_errors(word_clf, val_df["comments"], val_df["topic"])
err_df.head()

## 11. CLI

In [ ]:
import argparse

def run_cli():
    parser = argparse.ArgumentParser(description="Teacher vs Course classification")
    parser.add_argument("--use_sentiment_prefix", action="store_true")
    args = parser.parse_args(args=[])
    x_train = train_df["comments"] if not args.use_sentiment_prefix else sent_pref_train
    x_val = val_df["comments"] if not args.use_sentiment_prefix else sent_pref_val
    model = Pipeline([("tfidf", TfidfVectorizer(ngram_range=(1,2))), ("clf", LogisticRegression(max_iter=1000, class_weight="balanced"))])
    evaluate(model, x_train, train_df["topic"], x_val, val_df["topic"], "CLI run")

# run_cli()

---